In [1]:
# 1
!pip install natasha -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 67.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, Conv1D, GRU, LSTM, Dropout
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from collections import Counter
from nltk.corpus import stopwords
from string import punctuation
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from natasha import Doc, MorphVocab, Segmenter, NewsEmbedding, NewsMorphTagger

import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def natasha_lemmatize(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return " ".join(_.lemma for _ in doc.tokens)

Сформируем общий датасет на основе наборов для обучения и теста.

In [4]:
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/i/koR5eMCToCZS2Q'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('positive.csv', 'wb') as f:
    f.write(download_response.content)

public_key = 'https://disk.yandex.ru/i/koR5eMCToCZS2Q'

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']

# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('negative.csv', 'wb') as f:
    f.write(download_response.content)

In [5]:
# считываем данные и заполняем общий датасет
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)
df.reset_index(inplace=True)

<ipython-input-5-b4ad31c73284>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = positive.append(negative)


In [6]:
df['text'].sample()

129846    Спасибо за то,Что Ты Есть у Меня !!! Оставайся...
Name: text, dtype: object

In [7]:
natasha_lemmatize(df['text'][4])

'@ irina_dyshkant вот что значить страшилка : d но блин , посмотреть весь часть , у ты создаться ощущение , что автор курить что-то : d'

In [8]:
df['lemmatized_text'] = df['text'].apply(natasha_lemmatize)
df.sample()

,index,text,label,lemmatized_text
206650,91739,"сегодня был замечательный день, спасибо вам мо...",negative,"сегодня быть замечательный день , спасибо вы м..."


In [9]:
noise = stopwords.words('russian') + list(punctuation)
corpus = [token for tweet in df['lemmatized_text']
          for token in nltk.tokenize.word_tokenize(tweet) if token not in noise]
print(len(corpus))
corpus[:5]

2256546


['first_timee', 'школоть', 'поверь', 'самый', 'd']

In [10]:
freq_dict = Counter(corpus)
# freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
# list(freq_dict_sorted)[:10]
freq_dict.most_common(10)

[('t', 46144),
 ('http', 45048),
 ('co', 45000),
 ('rt', 44166),
 ('d', 36648),
 ('весь', 30578),
 ('``', 30378),
 ('это', 24534),
 ('...', 17172),
 ('3', 12436)]

In [11]:
len(freq_dict)

163000

In [12]:
most_frequent_tokens = [item[0] for item in freq_dict.most_common(162000)][:54000]
avg_frequent_tokens = [item[0] for item in freq_dict.most_common(162000)][54000:108000]
less_frequent_tokens = [item[0] for item in freq_dict.most_common(162000)][108000:]
most_frequent_tokens[:10]

['t', 'http', 'co', 'rt', 'd', 'весь', '``', 'это', '...', '3']

In [13]:
x_train, x_test, y_train, y_test = train_test_split(df['lemmatized_text'], df.label, random_state=42)

In [14]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=most_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

    negative       0.25      0.25      0.25     28614
    positive       0.25      0.25      0.25     28842

    accuracy                           0.25     57456
   macro avg       0.25      0.25      0.25     57456
weighted avg       0.25      0.25      0.25     57456



In [15]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=avg_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


              precision    recall  f1-score   support

    negative       0.00      0.00      0.00      6124
    positive       0.79      0.44      0.56     51332

    accuracy                           0.39     57456
   macro avg       0.39      0.22      0.28     57456
weighted avg       0.70      0.39      0.50     57456



In [16]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=less_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

    negative       0.00      0.00      0.00      6213
    positive       0.78      0.44      0.56     51243

    accuracy                           0.39     57456
   macro avg       0.39      0.22      0.28     57456
weighted avg       0.70      0.39      0.50     57456



In [17]:
# 2
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=most_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

    negative       0.25      0.25      0.25     28614
    positive       0.25      0.25      0.25     28842

    accuracy                           0.25     57456
   macro avg       0.25      0.25      0.25     57456
weighted avg       0.25      0.25      0.25     57456



In [18]:
coefs = clf.coef_[0]
top_coefs = np.argsort(coefs)[-20:]
top_coefs

array([21982, 25244, 22539, 10523,  8184, 25829, 19134, 21664, 24078,
       13062, 16432, 18730, 21353, 13228, 18109, 20562, 16850, 15646,
       13360, 12155])

In [19]:
tokens = list(vec.vocabulary_.items())
for _ in reversed(top_coefs):
    print(tokens[_])

('подтянуть', 12155)
('тульский', 13360)
('телескоп', 15646)
('брекет', 16850)
('tyorkinvassya', 20562)
('jas_yuu', 18109)
('забыться', 13228)
('katyalukinova', 21353)
('lilsh', 18730)
('гастрит', 16432)
('наехать', 13062)
('огрооомное', 24078)
('покидаться', 21664)
('гуру', 19134)
('накипеть', 25829)
('алилуя', 8184)
('похвала', 10523)
('вранье', 22539)
('nadyysha', 25244)
('valensy_shine', 21982)


In [20]:
vec = TfidfVectorizer(ngram_range=(1, 1), tokenizer=nltk.tokenize.word_tokenize, vocabulary=most_frequent_tokens)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


              precision    recall  f1-score   support

    negative       0.27      0.27      0.27     28710
    positive       0.27      0.26      0.27     28746

    accuracy                           0.27     57456
   macro avg       0.27      0.27      0.27     57456
weighted avg       0.27      0.27      0.27     57456



In [21]:
features_num = [2**19, 2**20, 2**21, 2**22]

for n_features in features_num:
    print(f'Number of features - {n_features}:')
    vec = HashingVectorizer(n_features=n_features,)
    bow = vec.fit_transform(x_train)
    clf = LogisticRegression(random_state=42, max_iter=500)
    clf.fit(bow, y_train)
    pred = clf.predict(vec.transform(x_test))
    print(classification_report(pred, y_test))

Number of features - 524288:
              precision    recall  f1-score   support

    negative       0.25      0.25      0.25     28594
    positive       0.25      0.25      0.25     28862

    accuracy                           0.25     57456
   macro avg       0.25      0.25      0.25     57456
weighted avg       0.25      0.25      0.25     57456

Number of features - 1048576:
              precision    recall  f1-score   support

    negative       0.25      0.25      0.25     28563
    positive       0.25      0.25      0.25     28893

    accuracy                           0.25     57456
   macro avg       0.25      0.25      0.25     57456
weighted avg       0.25      0.25      0.25     57456

Number of features - 2097152:
              precision    recall  f1-score   support

    negative       0.24      0.25      0.25     28561
    positive       0.25      0.25      0.25     28895

    accuracy                           0.25     57456
   macro avg       0.25      0.25      

In [22]:
vec = HashingVectorizer(n_features=2**19,)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

    negative       0.25      0.25      0.25     28594
    positive       0.25      0.25      0.25     28862

    accuracy                           0.25     57456
   macro avg       0.25      0.25      0.25     57456
weighted avg       0.25      0.25      0.25     57456



In [23]:
y_train_bin = np.where(y_train=='positive', 1, 0)
y_test_bin = np.where(y_test=='positive', 1, 0)

In [24]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train_bin))
valid_data = tf.data.Dataset.from_tensor_slices((x_test, y_test_bin))

In [25]:
train_data = train_data.batch(32)
valid_data = valid_data.batch(32)

In [26]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [27]:
def custom_standardization(input_data):

    return input_data

vocab_size = 15000
seq_len = 140

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [28]:
embedding_dim=250

model = Sequential([
    vectorize_layer,
    Embedding(vocab_size, embedding_dim),
    Conv1D(embedding_dim, 3),
    Conv1D(embedding_dim, 2),
    GRU(350),
    Dense(200, activation='relu'),
    Dense(1)
])

In [29]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-3)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [30]:
model.fit(train_data, validation_data=valid_data, epochs=2)

Epoch 1/2
5387/5387 [==============================] - 143s 24ms/step - loss: 0.6949 - accuracy: 0.4996 - val_loss: 0.6932 - val_accuracy: 0.5006
Epoch 2/2
5387/5387 [==============================] - 99s 18ms/step - loss: 0.6932 - accuracy: 0.4998 - val_loss: 0.6932 - val_accuracy: 0.5006


In [31]:
logits = model.predict(valid_data)
prediction = tf.round(tf.nn.sigmoid(logits))
class_preds = np.where(prediction==1, 'positive', 'negative')

1796/1796 [==============================] - 11s 6ms/step


In [32]:
print(classification_report(class_preds, y_test))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

    negative       1.00      0.50      0.67     57456
    positive       0.00      0.00      0.00         0

    accuracy                           0.50     57456
   macro avg       0.50      0.25      0.33     57456
weighted avg       1.00      0.50      0.67     57456



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
